In [1]:
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.io import votable, ascii
# import h5py
import json
import numpy as np
from types import SimpleNamespace

### SQL query on MAST

In [3]:
def z_cat_query(tl, br):
    string = "select * from catalogRecordRowStore where raMean>{:.2f} and raMean<{:.2f} and decMean>{:.2f} and decMean<{:.2f}".format(br[0], tl[0], br[1], tl[1])
    return string

In [4]:
def sexa_to_deci(c):
    sc = SkyCoord(c, unit=(u.hourangle, u.deg))
    return sc.ra.deg, sc.dec.deg

In [5]:
br = sexa_to_deci("00:03:19.1890576 +09:54:13.899849")
tl = sexa_to_deci("00:04:35.3301991 +10:12:15.172584")

In [6]:
print("test<{:.2f}".format(br[0]))

test<0.83


In [7]:
print(z_cat_query(tl, br))

select * from catalogRecordRowStore where raMean>0.83 and raMean<1.15 and decMean>9.90 and decMean<10.20


### Preparing prioritize.py input

In [31]:
center = "00:03:48.8888534 +10:04:10.737999"
# center = "00:03:50.0443707 +10:04:22.110113"
# guide = 1237678906782974123
# align = []
print(sexa_to_deci(center))

(0.9537035558333332, 10.069649444166666)


In [32]:
sc = SkyCoord(center, unit=(u.hourangle, u.deg))
sc.ra.hour

0.06358023705555556

The following cells are just a test.

In [18]:
archivalz = ascii.read('archivalz.csv')
list(archivalz['RA'].data)

In [20]:
v = SimpleNamespace()

In [26]:
io_dir = '1/'
align = "{}align.xml".format(io_dir)
try:
    votb2 = (votable.parse_single_table(source=align)).to_table()
    v.aligninfo = list(votb2.iterrows('_RAJ2000','_DEJ2000','rmag'))
except:
    raise SystemExit('No align file.')

In [22]:
votb2.columns

<TableColumns names=('_RAJ2000','_DEJ2000','_V','mode','q_mode','cl','SDSS8','m_SDSS8','Im','RA_ICRS','DE_ICRS','zsp','umag','e_umag','gmag','e_gmag','rmag','e_rmag','imag','e_imag','zmag','e_zmag','Q','ObsDate','objID')>

In [23]:
print(list(votb2.iterrows('_RAJ2000','_DEJ2000','rmag')))

[(0.948284, 9.962229, 15.765), (1.001962, 9.969801, 19.387), (0.875299, 10.185863, 15.263), (0.846498, 10.140442, 15.871), (0.859229, 10.16519, 18.891)]


In [24]:
v.aligninfo

[(0.948284, 9.962229, 15.765),
 (1.001962, 9.969801, 19.387),
 (0.875299, 10.185863, 15.263),
 (0.846498, 10.140442, 15.871),
 (0.859229, 10.16519, 18.891)]

### Downloading the PS catalog

In [9]:
import ps_api

In [10]:
# cl_pos = ps_api.resolve('RMJ000343.8+100123.8')
cl_pos = sexa_to_deci(center)
cl_pos

(0.9537035558333332, 10.069649444166666)

In [11]:
# radius = (9 * u.arcmin).to(u.deg).value
radius = (12 * u.arcmin).to(u.deg).value
# radius = .03

In [20]:
# constraints = {'nDetections.gt':1, 'rMeanKronMag.gt':-990}
# constraints = {'rMeanKronMag.gt':-990}
constraints = {}

In [21]:
columns = """objID,raMean,decMean,rMeanKronMag""".split(',')
# strip blanks and weed out blank and commented-out values
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]

In [22]:
results = ps_api.ps1cone(*cl_pos, radius, columns=columns, **constraints)
# try including release="dr2" as an argument

In [23]:
with open('ps_api_res.csv', 'w') as f:
    f.write(results)